In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from scipy.stats import randint
from sklearn.utils import resample
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
import joblib


/Users/tianqingzhou/opt/anaconda3/envs/ai_env/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/tianqingzhou/opt/anaconda3/envs/ai_env/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# dataset_path = '/content/gdrive/MyDrive/Group project dataset/diabetes_012_health_indicators_BRFSS2015.csv'
dataset_path = '/Users/tianqingzhou/Documents/GeorgiaTech/CS6440_IntroHealthInformatics/CS6440-Project/kaggle_data/diabetes_012_health_indicators_BRFSS2015.csv'

In [3]:
dataset = pd.read_csv(dataset_path)
dataset

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253675,0.0,1.0,1.0,1.0,45.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,5.0,0.0,1.0,5.0,6.0,7.0
253676,2.0,1.0,1.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,0.0,1.0,0.0,11.0,2.0,4.0
253677,0.0,0.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0
253678,0.0,1.0,0.0,1.0,23.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,7.0,5.0,1.0


In [5]:
# see the percentage of 3 classes
class_counts = dataset['Diabetes_012'].value_counts()
class_percentages = dataset['Diabetes_012'].value_counts(normalize=True) * 100

# Combine both counts and percentages into a single DataFrame
class_summary = pd.DataFrame({
    'Count': class_counts,
    'Percentage (%)': class_percentages
})

print(class_summary)

               Count  Percentage (%)
Diabetes_012                        
0.0           213703       84.241170
2.0            35346       13.933302
1.0             4631        1.825528


In [4]:
# Combine the classes 1 and 2 in the original dataframe
dataset['Diabetes_012'] = dataset['Diabetes_012'].apply(lambda x: 0 if x == 0 else 1)  # Re-label class 1 and 2 as 1, keep class 0 as 0

In [5]:
# check if class 1 and 2 have been merged successfully

class_counts = dataset['Diabetes_012'].value_counts()
class_percentages = dataset['Diabetes_012'].value_counts(normalize=True) * 100

# Combine both counts and percentages into a single DataFrame
class_summary = pd.DataFrame({
    'Count': class_counts,
    'Percentage (%)': class_percentages
})

print(class_summary)

               Count  Percentage (%)
Diabetes_012                        
0             213703        84.24117
1              39977        15.75883


In [6]:
dataset.columns

Index(['Diabetes_012', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')

In [7]:
X = dataset.drop(columns=['Diabetes_012'])
y = dataset['Diabetes_012']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [11]:
y_train.value_counts()

Diabetes_012
0    170908
1     32036
Name: count, dtype: int64

In [8]:
int(y_train.value_counts()[0] / y_train.value_counts()[1])

5

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def model_evaluation(y_test, y_pred):
    """
    model_name: string to show model

    """
    recall = recall_score(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    conf_matrix = confusion_matrix(y_test, y_pred)

    conf_matrix_df = pd.DataFrame(
        conf_matrix,
        index=["Actual Not Fraud", "Actual Fraud"],
        columns=["Predicted Not Fraud", "Predicted Fraud"]
    )
    # print((tn, fp, fn, tp))

    print(f"Recall: {recall:.2f}")
    print(f"Accuracy: {acc:.2f}")
    print(f"Precision: {prec:.2f}")    
    print(f"F1 Score: {f1:.2f}")
    print(f"conf_matrix: {conf_matrix_df}")

    return recall, acc, prec, f1,conf_matrix

In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

# Oversample the minority class
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
# # Split data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize and train the XGBClassifier
xgb = XGBClassifier(
    n_estimators=300, max_depth=6, learning_rate=0.1, 
    random_state=42, use_label_encoder=False, 
    eval_metric='auc', 
    #scale_pos_weight = int(y_train.value_counts()[0] / y_train.value_counts()[1]) # handle the data imbalance
    objective = 'binary:logistic'
    )
xgb.fit(X_resampled, y_resampled)

# Make predictions on the test set
y_pred = xgb.predict(X_test)


# Evaluate the model use the customized function
# Here the recall is 0.69, which is much higher than logistic regression
recall, acc, prec, f1,conf_matrix = model_evaluation(y_test, y_pred)
print(pd.DataFrame(conf_matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]))
# Evaluate model performance
print("Classification Report:")
print(classification_report(y_test, y_pred))

/Users/tianqingzhou/opt/anaconda3/envs/ai_env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [19:56:46] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Recall: 0.23
Accuracy: 0.85
Precision: 0.58
F1 Score: 0.33
conf_matrix:                   Predicted Not Fraud  Predicted Fraud
Actual Not Fraud                41464             1331
Actual Fraud                     6101             1840
          Predicted 0  Predicted 1
Actual 0        41464         1331
Actual 1         6101         1840
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.97      0.92     42795
           1       0.58      0.23      0.33      7941

    accuracy                           0.85     50736
   macro avg       0.73      0.60      0.62     50736
weighted avg       0.83      0.85      0.83     50736



In [31]:
import pickle

# Save the trained model to a pickle file
with open('xgb_model.pkl', 'wb') as file:
    pickle.dump(xgb, file)

print("Model saved to xgb_model.pkl")


Model saved to xgb_model.pkl


In [51]:
X_test.iloc[1,:]

HighBP                   1.0
HighChol                 1.0
CholCheck                1.0
BMI                     28.0
Smoker                   0.0
Stroke                   0.0
HeartDiseaseorAttack     0.0
PhysActivity             1.0
Fruits                   1.0
Veggies                  1.0
HvyAlcoholConsump        0.0
AnyHealthcare            1.0
NoDocbcCost              0.0
GenHlth                  3.0
MentHlth                 0.0
PhysHlth                 0.0
DiffWalk                 0.0
Sex                      0.0
Age                     13.0
Education                6.0
Income                   6.0
Name: 132821, dtype: float64

In [52]:
# Load the model from the pickle file
with open('xgb_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Verify the model is loaded by making a prediction
y_pred = loaded_model.predict(X_test.iloc[1:2,])
print("Model loaded successfully and used for predictions.")
print(y_pred)


Model loaded successfully and used for predictions.
[0]


In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from xgboost import XGBClassifier

# # Split data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize and train the XGBClassifier
xgb = XGBClassifier(
    n_estimators=300, max_depth=10, learning_rate=0.1, 
    random_state=42, use_label_encoder=False, 
    eval_metric='logloss', 
    scale_pos_weight = int(y_train.value_counts()[0] / y_train.value_counts()[1]) # handle the data imbalance
    )
xgb.fit(X_train, y_train)

# Make predictions on the test set
y_proba = xgb.predict_proba(X_test)[:,1]
custom_threshold = 0.7  # Example value, tune based on your evaluation
y_pred_custom = (y_proba >= custom_threshold).astype(int)


# Evaluate the model use the customized function
# Here the recall is 0.69, which is much higher than logistic regression
recall, acc, prec, f1,conf_matrix = model_evaluation(y_test, y_pred_custom)
print(pd.DataFrame(conf_matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]))
# Evaluate model performance
print("Classification Report:")
print(classification_report(y_test, y_pred_custom))

/Users/tianqingzhou/opt/anaconda3/envs/ai_env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [19:51:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Recall: 0.40
Accuracy: 0.83
Precision: 0.47
F1 Score: 0.43
conf_matrix:                   Predicted Not Fraud  Predicted Fraud
Actual Not Fraud                39205             3590
Actual Fraud                     4787             3154
          Predicted 0  Predicted 1
Actual 0        39205         3590
Actual 1         4787         3154
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.92      0.90     42795
           1       0.47      0.40      0.43      7941

    accuracy                           0.83     50736
   macro avg       0.68      0.66      0.67     50736
weighted avg       0.82      0.83      0.83     50736



In [8]:
# Separate the majority and minority classes
majority_data = dataset[dataset['Diabetes_012'] == 0]
minority_data = dataset[dataset['Diabetes_012'] == 1]

In [9]:
# Downsample majority_data to match the size of minority_data
majority_data_downsampled = resample(majority_data, replace=False, n_samples=len(minority_data), random_state=42)

In [10]:
# Combine the downsampled majority_data samples with all minority_data samples
balanced_df = pd.concat([majority_data_downsampled, minority_data]).reset_index(drop=True)

In [11]:
# Shuffle the new balanced DataFrame
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [12]:
# Check the new distribution of Y values
print(balanced_df['Diabetes_012'].value_counts())

Diabetes_012
0    39977
1    39977
Name: count, dtype: int64


In [13]:
# split the dataset into X (featuers) and Y (label)
X = balanced_df.drop(columns=['Diabetes_012'])  # Features
Y = balanced_df['Diabetes_012']  # Labels

In [14]:
# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# Apply SMOTE only on the training data
# smote = SMOTE(random_state=42)
# X_train_smote, Y_train_smote = smote.fit_resample(X_train, Y_train)

In [15]:
# Normalize the dataset
scaler = StandardScaler()

# Fit the scaler on the training data and transform it
X_train = scaler.fit_transform(X_train)

# Transform the test data using the same scaler
X_test = scaler.transform(X_test)

In [ ]:
# Model 1: Logistic regression

In [16]:
# Initialize and train Logistic Regression with balanced class weights
log_reg = LogisticRegression(class_weight='balanced', random_state=42)


In [17]:
# Hyperparameter tuning using GridSearchCV
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularization strength
    'solver': ['lbfgs', 'liblinear'],  # Optimization algorithm
    'max_iter': [100, 200, 300]  # Maximum iterations
}

grid_search = GridSearchCV(log_reg, param_grid, cv=5, verbose=1, n_jobs=-1)


In [ ]:
# Fit the grid search to the data
grid_search.fit(X_train, Y_train)

In [34]:
# Check original training data size
print("Original X_train size:", X_train.shape)
print("Original Y_train class distribution:\n", Y_train.value_counts())

# Check SMOTE-applied data size
# print("SMOTE-applied X_train size:", X_train_smote.shape)
# print("SMOTE-applied Y_train class distribution:\n", pd.Series(Y_train_smote).value_counts())


Original X_train size: (63963, 21)
Original Y_train class distribution:
 Diabetes_012
0    32110
1    31853
Name: count, dtype: int64


In [35]:
# Print the best hyperparameters found by GridSearchCV
print("Best hyperparameters found by GridSearchCV:")
print(grid_search.best_params_)

Best hyperparameters found by GridSearchCV:
{'C': 0.01, 'max_iter': 100, 'solver': 'lbfgs'}


In [36]:
# Train the best Logistic Regression model from the grid search
best_log_reg = grid_search.best_estimator_

In [37]:
# Make predictions on the test set
Y_pred = best_log_reg.predict(X_test)

In [ ]:
# Evaluate the model
print("Classification Report:")
print(classification_report(Y_test, Y_pred))

In [ ]:
# Model 2: Random Forest model

In [19]:
# Initialize the Random Forest model with balanced class weights
rf_model = RandomForestClassifier(random_state=42, class_weight='balanced')

In [20]:
# Define the parameter distribution
param_dist = {
    'n_estimators': randint(50, 500),
    'max_depth': [10, 20, 30, None],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 4),
    'max_features': ['sqrt', 'log2']
}


In [21]:
# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_dist, n_iter=50,
                                   cv=5, scoring='accuracy', n_jobs=-1, verbose=2, random_state=42)


In [22]:
# Fit the random search to the data
random_search.fit(X_train, Y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


RandomizedSearchCV(cv=5,
                   estimator=RandomForestClassifier(class_weight='balanced',
                                                    random_state=42),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 30, None],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7bee86f77400>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7bee86b49f00>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7bee86f75030>},
                   random_state=42, scoring='accuracy', verbose=2)

In [23]:
# Get the best model from grid search
best_rf_model = random_search.best_estimator_
print(best_rf_model)

RandomForestClassifier(class_weight='balanced', max_depth=20,
                       min_samples_leaf=3, n_estimators=276, random_state=42)


In [24]:
# Make predictions on the test set
Y_pred_rf = best_rf_model.predict(X_test)

In [25]:
# Evaluate the model
print("Best Hyperparameters:", random_search.best_params_)
print("Random Forest Classification Report:")
print(classification_report(Y_test, Y_pred_rf))
print("Accuracy:", accuracy_score(Y_test, Y_pred_rf))

Best Hyperparameters: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 276}
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.70      0.73      7867
           1       0.73      0.78      0.76      8124

    accuracy                           0.74     15991
   macro avg       0.75      0.74      0.74     15991
weighted avg       0.75      0.74      0.74     15991

Accuracy: 0.744356200362704


In [27]:
# Define the final model
final_model = RandomForestClassifier(class_weight='balanced', max_depth=20,max_features='sqrt', min_samples_leaf=3, min_samples_split=2, n_estimators=276, random_state=42)

In [28]:
# Create the final model by fitting the entire dataset
final_model.fit(X,Y)

RandomForestClassifier(class_weight='balanced', max_depth=20,
                       min_samples_leaf=3, n_estimators=276, random_state=42)

In [31]:
# Export the final model as a pkl file
joblib.dump(final_model,'ML_model.pkl')

['ML_model.pkl']

In [30]:
# import os
# os.listdir()

['.config', 'ML model', 'gdrive', 'sample_data']

In [32]:
# Export the list of features as a pkl file
joblib.dump(list(X.columns),'feature_names.pkl')

['feature_names.pkl']

In [35]:
# Reload model to test predictions
Diabetes_model = joblib.load('ML_model.pkl')

In [72]:
# Test model predication against the true value
print(Diabetes_model.predict(X.iloc[28666].values.reshape(1, -1)))
print(Y.iloc[28666])

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([0])

In [ ]:
# Random forest model with gridsearch instead of random search

In [ ]:
# Set up hyperparameters for grid search
param_grid = {
    'n_estimators': [50, 100, 200],       # Number of trees in the forest
    'max_depth': [5, 10, 20],      # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],      # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]
    'max_features': ['sqrt', 'log2'] # Minimum number of samples required to be at a leaf node
}

In [ ]:
# Set up GridSearchCV
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid,
                               scoring='f1_weighted', cv=5, n_jobs=1, verbose=2)

In [ ]:
# Fit the model to the training data
grid_search_rf.fit(X_train, Y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=  28.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=  26.7s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=  27.7s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=  27.5s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=  28.0s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  59.5s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  56.8s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  59.7s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  57.2s
[CV] END max_depth=5, min_samples_leaf=1, m

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=42),
             n_jobs=1,
             param_grid={'max_depth': [5, 10, 20], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [50, 100, 200]},
             scoring='f1_weighted', verbose=2)

In [ ]:
# Get the best model from grid search
best_rf_model = grid_search_rf.best_estimator_


In [ ]:
# Make predictions on the test set
Y_pred_rf = best_rf_model.predict(X_test)

In [ ]:
# Evaluate the model
print("Best Hyperparameters:", grid_search_rf.best_params_)
print("Random Forest Classification Report:")
print(classification_report(Y_test, Y_pred_rf))
print("Accuracy:", accuracy_score(Y_test, Y_pred_rf))

Best Hyperparameters: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Random Forest Classification Report:
              precision    recall  f1-score   support

         0.0       0.88      0.94      0.91     42795
         1.0       0.12      0.00      0.00       944
         2.0       0.46      0.34      0.39      6997

    accuracy                           0.84     50736
   macro avg       0.49      0.43      0.44     50736
weighted avg       0.81      0.84      0.82     50736

Accuracy: 0.8397981709239988


In [ ]:
# Model 3: XGBoost model

In [ ]:
# Initialize the XGBoost model
xgb_model = XGBClassifier(objective='multi:softmax', eval_metric='mlogloss')

In [ ]:
# Define a smaller parameter grid for RandomizedSearch
param_dist = {
    'learning_rate': np.linspace(0.01, 0.3, 5),  # Smaller range, fewer steps
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2]  # Narrowed values for quicker testing
}

In [ ]:
# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=20,  # Number of parameter combinations to try
    scoring='accuracy',
    cv=3,  # Reduced from 5 to 3 folds
    n_jobs=-1,
    verbose=2,
    random_state=42  # Set for reproducibility
)

In [ ]:
# Fit the randomized search to the data
random_search.fit(X_train, Y_train)


Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [00:17:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='mlogloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learnin...
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           objective='multi:softmax', ...),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.8, 1.0],
                                        'gamma': [0, 0.1, 0.2],
                                        'learning_rate': array([0.01  , 0.0825, 0.155 , 0.2275, 0.3   ]),
                                        'max_depth': [3, 5, 7],
                                        'n_estimators': [50, 100, 200],
                                        'subsample': [0.8, 1.0]},
                   random_state=42, scoring='accuracy', verbose=2)

In [ ]:
# Print best parameters and the corresponding score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)


Best Parameters: {'subsample': 1.0, 'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.22749999999999998, 'gamma': 0.2, 'colsample_bytree': 1.0}
Best Score: 0.8673789407166429


In [ ]:
# Predict using the best model
best_xgb_model = random_search.best_estimator_
Y_pred = best_xgb_model.predict(X_test)


In [ ]:
# Evaluate model performance
print("Classification Report:")
print(classification_report(Y_test, Y_pred))


Classification Report:
              precision    recall  f1-score   support

         0.0       0.87      0.97      0.92     42795
         1.0       0.00      0.00      0.00       944
         2.0       0.52      0.23      0.32      6997

    accuracy                           0.85     50736
   macro avg       0.46      0.40      0.41     50736
weighted avg       0.80      0.85      0.82     50736



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Model 4: Support Vector Machine

In [ ]:
# Initialize SVM model with 'balanced' class weight
svm_model = SVC(class_weight='balanced', random_state=42)

In [ ]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10, 100],       # Regularization parameter
    'kernel': ['linear', 'rbf'],   # Type of kernel
    'gamma': ['scale', 'auto']     # Kernel coefficient
}


In [ ]:
# Set up GridSearchCV with cross-validation
grid_search_svm = GridSearchCV(estimator=svm_model, param_grid=param_grid,
                               scoring='f1_weighted', cv=5, n_jobs=2, verbose=2)

In [ ]:
# Fit the model
grid_search_svm.fit(X_train, Y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [ ]:
# Step 5: Make predictions on the test set
Y_pred = grid_search_svm.predict(X_test)

In [ ]:
# Step 6: Evaluate the model
print("Best parameters found by GridSearchCV:", grid_search_svm.best_params_)
print("Classification Report:")
print(classification_report(Y_test, Y_pred))
print("Accuracy Score:", accuracy_score(Y_test, Y_pred))
